In [2]:
import os
import matplotlib.pyplot as plt
import IPython.display as ipd

import torch
from torch import nn, optim

from model import Model
from train import train_model
from data import get_dataloaders
from utils import num_params, label_to_index

In [ ]:
batch_size = 512
validation = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'

dataloaders = get_dataloaders(batch_size, validation, device)
dataloaders

In [4]:
# torch.multiprocessing.set_start_method('spawn')

for tensors, targets in dataloaders['train']:
    tensor = tensors[0]
    target = targets[0]
    break

In [ ]:
plt.plot(tensor.t().numpy())
plt.grid()
plt.show()

In [ ]:
ipd.Audio(tensor, rate=16000)

In [ ]:
cfg = [32, 'M', 64, 'M', 128, 'M', 256, 'M', 512]
model = Model(1, 35, cfg, 3)
print('Trainable parameters: {}'.format(num_params(model)))

In [8]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5,
    patience=5)

In [ ]:
model, (losses, accuracies) = train_model(model, criterion, optimizer, scheduler,
    dataloaders, epochs=50, scheduler_basis='val', save_checkpoint=True,
    save_checkpoint_filepath='checkpoint.pth.tar', load_checkpoint=False,
    load_checkpoint_filepath='checkpoint.pth.tar', device=device)

In [ ]:
plt.plot(range(1, 51), losses['train'], label='training loss')
plt.plot(range(1, 51), losses['val'], label='validation loss')
plt.grid()
plt.legend(loc='best')
plt.xlim([1, 50])
plt.ylim([-1, 0])
plt.show()

In [ ]:
plt.plot(range(1, 51), accuracies['train'], label='training accuracy')
plt.plot(range(1, 51), accuracies['val'], label='validation accuracy')
plt.grid()
plt.legend(loc='best')
plt.xlim([1, 50])
plt.ylim([0, 1])
plt.show()